In [3]:
import re
import os
import glob
from tkinter import *
from urduhack import *
import subprocess


In [4]:
stopwords = []

def get_all_headlines(folder_path):
    extraWordsinHeadlines = ['.doc','Urdu NEWS dataset//','voa','bbc','dataset','entertainment','sports','miscleneous','politics','//',"'"]
    headlines = []
    print(folder_path)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            print(file_path)
            for word in extraWordsinHeadlines:
                file_path = file_path.replace(word,' ')
            headlines.append(file_path.strip())
    return headlines

def splitwords(headlines):
    lugat = []
    for headline in headlines:
        alfaz = re.split('\W+',headline)
        lugat.append(alfaz)
    return lugat


def readstopwords(file_name):
    global stopwords
    with open(file_name, "r",encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    return stopwords


def removeStopWords(words):
    #106033 -> 4571 after stopwords removal
    file_name = 'StopWordsUrduList.txt'
    stopwords = readstopwords(file_name)
    wordswithoutstopwords = []
    for outer in words:
        for word in outer:
            if (word not in stopwords or word != '') and (word not in wordswithoutstopwords):
                wordswithoutstopwords.append(word)
    return wordswithoutstopwords
                
    
def relatedNews(dataset,inputnews):
    relatedNewsList = []
    t = 0.5
    datasetlist = dataset
    ni = inputnews
    sizeofdatasetlist = len(datasetlist)
    for j in range(0,sizeofdatasetlist):
        nj = datasetlist[j]
        Sij = getSimilarityScore(ni,nj)
        if Sij >= t:
            relatedNewsList.append(nj)
    print(relatedNewsList)
    return relatedNewsList
    
    

def getTokensList(n):
    wordsWoSW = []
    words = []
    words = normalize(n)
    words = re.split('\W+',words)
    for word in words:
        if word in stopwords:
            continue
        wordsWoSW.append(word)
        
    return wordsWoSW

    
    
def getSimilarityScore(ni,nj):
    tli = getTokensList(ni)
    tlj = getTokensList(nj)
    sti = len(tli)
    stj = len(tlj)
    Sij = 0
    mij = 0
    
    for x in range(0,sti):
        for y in range(0,stj):
            if tli[x] == tlj[y]:
                print(tli[x])
                mij = mij + 1
    
    if mij > 0:
        avg = (sti + stj)/2
        Sij = mij / avg
    
    return Sij


In [5]:
# example usage
inputnews = input("Enter News")
folder_path = 'Urdu NEWS dataset/'
headlines = get_all_headlines(folder_path)
words = splitwords(headlines)
refinedwords = removeStopWords(words)
cluster = relatedNews(headlines,inputnews)

Enter Newsداعش نے موصل میں شہریوں کو 'انسانی ڈھال بنا رکھا
Urdu NEWS dataset/
Urdu NEWS dataset/voa dataset/misc/کتا کینسر سے پیشگی خبردار کر سکتا ہے.doc
Urdu NEWS dataset/voa dataset/misc/درگاہ خواجہ نظام الدین اولیا کے سجادہ نشین پاکستان میں لاپتا ہو گئے .doc
Urdu NEWS dataset/voa dataset/misc/تین افراد کے پہلے بچے کی پیدائش.doc
Urdu NEWS dataset/voa dataset/misc/دوسروں کے لیے جینا !!.doc
Urdu NEWS dataset/voa dataset/misc/معروف پاکستانی کرکٹر امتیاز احمد علیل ہوگئے.doc
Urdu NEWS dataset/voa dataset/misc/’ایڈز سے نفرت کریں، مریض سے نہیں‘.doc
Urdu NEWS dataset/voa dataset/misc/ٹینیسی امریکی نیوکلیئر پاور پلانٹ بجلی پیدا کرنے لگا.doc
Urdu NEWS dataset/voa dataset/misc/پاکستان ریلوے کا نیا دور، ای ٹکٹنگ کی سہولت متعارف  .doc
Urdu NEWS dataset/voa dataset/misc/بھوک کے شکار 118 ملکوں میں پاکستان 107 نمبر پر.doc
Urdu NEWS dataset/voa dataset/misc/مسلمان ایتھلیٹس، ’نائیکی‘ حجاب پر مبنی لباس متعارف کرائے گا رپورٹ.doc
Urdu NEWS dataset/voa dataset/misc/سمندری طوفان 'ہرمائن' فلوریڈا میں داخل.d

شہریوں
انسانی
داعش
داعش
داعش
موصل
داعش
موصل
داعش
شہریوں
داعش
داعش
موصل
داعش
انسانی
داعش
موصل
موصل
موصل
شہریوں
شہریوں
داعش
انسانی
داعش
داعش
داعش
موصل
موصل
داعش
موصل
شہریوں
انسانی
ڈھال
بنا
داعش
موصل
داعش
داعش
داعش
موصل
داعش
داعش
موصل
انسانی
داعش
شہریوں
داعش
انسانی
داعش
موصل
موصل
انسانی
بنا
داعش
انسانی
موصل
داعش
داعش
موصل
شہریوں
داعش
انسانی
انسانی
بنا
انسانی
انسانی
بنا
بنا
موصل
موصل
شہریوں
شہریوں
داعش
موصل
موصل
موصل
موصل
داعش
موصل
انسانی
ڈھال
داعش
داعش
موصل
انسانی
موصل
موصل
انسانی
داعش
بنا
موصل
موصل
['Urdu NEWS  /   / /’داعش نے موصل میں شہریوں کو  انسانی ڈھال بنا رکھا ہے‘', 'Urdu NEWS  /   / /’داعش موصل میں انسانی ڈھال استعمال کر رہی ہے‘']
